In [ ]:
import sys
print(sys.version)


In [2]:
# Librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [4]:
# Abrir los graficos en el navegador

pio.renderers.default = 'browser'

In [6]:
# Carga de datos

df_ple7= pd.read_csv(r"../data/9-1-2026/data-1768216236753.csv")
df_ple1 = pd.read_csv(r"../data/9-1-2026/data-1768216350513.csv")
# df_data_raw = pd.read_csv(r"../data/data-raw30122025.csv", encoding='latin1')

df_ple1['maquina'] = 'Plegadora 1'
df_ple7['maquina'] = 'Plegadora 7'

# Ordenamiento de los datos en base a su marca temporal
df_ple1 = df_ple1.sort_values('temporal_placa').reset_index(drop=True)  
df_ple7 = df_ple7.sort_values('temporal_placa').reset_index(drop=True)


In [ ]:
# Manejo de data-raw (datos ingresados por los operarios de ARSEMET)
import pandas as pd

df_data_raw.columns = (
    df_data_raw.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
)

# Filtrar registros con texto 'PLEGADORA'
df_data = df_data_raw[
    df_data_raw['descripcion']
        .str.upper()
        .str.contains('PLEGADORA', na=False)
].copy()

# Extraer número de plegadora
df_data['nro_plegadora'] = (
    df_data['descripcion']
    .str.extract(r'PLEGADORA\s+(\d+)', expand=False)
)

# Quedarse solo con plegadora 1 y 7
df_data = df_data[df_data['nro_plegadora'].isin(['1', '7'])].copy()

df_data['maquina'] = 'Plegadora ' + df_data['nro_plegadora']

df_data[['descripcion', 'maquina']].drop_duplicates()


In [ ]:
# Calculo inicio estimado del ciclo

# Cantidad de golpes por pieza
df_data['ple_nro_golpes'] = pd.to_numeric(
    df_data['ple_nro_golpes'],
    errors='coerce'
)

# Cantidad de golpes por ciclo
df_data['golpes_ciclo'] = (
    df_data['ple_nro_golpes']*df_data['cant_buenas']
)

# Duración del ciclo en minutos
df_data['duracion_minutos'] = pd.to_numeric(
    df_data['tiempo_proc'],
    errors='coerce'
)

# Fecha de finalizacion (fecha_user)
df_data['fecha_fin'] = pd.to_datetime(
    df_data['fecha_usuario'],
    format='%d/%m/%Y %H:%M:%S:%f',
    errors='coerce'
)

df_data['fecha_inicio'] = (
    df_data['fecha_fin']
    - pd.to_timedelta(df_data['duracion_minutos'], unit='m')
)

# Ordenar cronológicamente por máquina
df_data = df_data.sort_values(
    by=['maquina', 'fecha_inicio']
).reset_index(drop=True)

# Crear un ID de ciclo por máquina
df_data['ciclo_id'] = (
    df_data
    .groupby('maquina')
    .cumcount() + 1
)

# DataFrame final de ciclos
df_ciclos = df_data[[
    'maquina',
    'ciclo_id',
    'ple_nro_golpes',
    'golpes_ciclo',
    'fecha_inicio',
    'fecha_fin',
    'duracion_minutos'
]].copy()

df_ciclos.head()


In [ ]:
# Verificacion
df_ciclos = df_ciclos.sort_values(
    by=['maquina', 'fecha_inicio']
)

df_ciclos


In [12]:
# Funcion preparar_df
# Esta funcion toma los datos de la plegadora y asigna a cada fila el turno correspondiente 

def preparar_df(df):
    df = df.copy()

    # Timestamp
    df['temporal_placa'] = pd.to_datetime(df['temporal_placa'])
    df['hora'] = df['temporal_placa'].dt.hour
    df['minuto'] = df['temporal_placa'].dt.minute

    # Turnos
    def asignar_turno(hora, minuto):
        t = hora * 60 + minuto

        # Pausas
        if 12*60 <= t < 12*60 + 30:
            return 'ALMUERZO'
        if 22*60 <= t < 22*60 + 30:
            return 'CENA'

        # Turnos
        if 5*60 <= t < 17*60:
            return 'TURNO MAÑANA'
        if 17*60 <= t < 22*60:
            return 'TURNO TARDE'
        if (t >= 22*60 + 30) or (t < 1*60):
            return 'TURNO TARDE'

        return 'FUERA_TURNO'

    df['turno'] = df.apply(
        lambda x: asignar_turno(x['hora'], x['minuto']),
        axis=1
    )
    
    # Potencias totales por timestamp
    df['p_activa_total'] = (
        df['potencia_a_r'] +
        df['potencia_a_s'] +
        df['potencia_a_t']
    )

    df['q_reactiva_total'] = (
        df['potencia_r_r'] +
        df['potencia_r_s'] +
        df['potencia_r_t']
    )

    return df


In [15]:
# Ajuste datos de las plegadoras

df_ple1 = preparar_df(df_ple1)
df_ple7 = preparar_df(df_ple7)

df_all = pd.concat([df_ple1, df_ple7], ignore_index=True)

df_all = df_all.sort_values(['maquina','temporal_placa']).reset_index(drop=True)

# Filtra los datos correspondientes al 10-1-2026
import datetime as dt

dia = dt.date(2026, 1, 9)
df_all = df_all[df_all['temporal_placa'].dt.date == dia]

In [13]:
# Visualización corrientes 
import plotly.graph_objects as go
import pandas as pd

# Horarios
fecha = df_all['temporal_placa'].dt.normalize().min()

horarios = {
    'TURNO_MAÑANA': (fecha + pd.Timedelta(hours=5),
                     fecha + pd.Timedelta(hours=17)),
    'ALMUERZO':     (fecha + pd.Timedelta(hours=12),
                     fecha + pd.Timedelta(hours=12, minutes=30)),
    'TURNO_TARDE':  (fecha + pd.Timedelta(hours=17),
                     fecha + pd.Timedelta(hours=22)),
    'CENA':         (fecha + pd.Timedelta(hours=22),
                     fecha + pd.Timedelta(hours=22, minutes=30)),
    'FUERA_TURNO':  (fecha + pd.Timedelta(hours=1),
                     fecha + pd.Timedelta(hours=5))
}

colores_turnos = {
    'TURNO_MAÑANA': 'rgba(52, 101, 164, 0.25)',
    'TURNO_TARDE':  'rgba(46, 139, 87, 0.25)',
    'ALMUERZO':     'rgba(218, 165, 32, 0.30)',
    'CENA':         'rgba(178, 34, 34, 0.30)',
    'FUERA_TURNO':  'rgba(105, 105, 105, 0.20)'
}

# Fases / máquinas
fases = {
    'Fase R': 'corriente_r',
    'Fase S': 'corriente_s',
    'Fase T': 'corriente_t'
}

maquinas = list(df_all['maquina'].unique())

# Figura y trazas
fig = go.Figure()

labels = []  # para el dropdown

for maq in maquinas:
    g = df_all[df_all['maquina'] == maq]
    for fase_label, fase_col in fases.items():
        labels.append(f'{maq} – {fase_label}')

        fig.add_trace(
            go.Scatter(
                x=g['temporal_placa'],
                y=g[fase_col],
                mode='lines',
                name=f'{maq} – {fase_label}',
                visible=False
            )
        )

# Estado inicial
fig.data[0].visible = True

# Dropdown único
buttons = []

for i, label in enumerate(labels):
    visible = [False] * len(fig.data)
    visible[i] = True

    buttons.append(
        dict(
            label=label,
            method='update',
            args=[
                {'visible': visible},
                {'title.text': f'Corriente vs tiempo – {label}'}
            ]
        )
    )

# Layout
fig.update_layout(
    title=dict(
        text=f'Corriente vs tiempo – {labels[0]}',
        x=0.5,
        xanchor='center',
        font=dict(size=18)
    ),
    yaxis_title='Corriente [A]',
    showlegend=False,
    margin=dict(r=260),
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=1.02,
            y=1.00,
            xanchor='left',
            yanchor='top'
        )
    ]
)

# Bandas horarias
for nombre, (inicio, fin) in horarios.items():
    fig.add_vrect(
        x0=inicio,
        x1=fin,
        fillcolor=colores_turnos[nombre],
        opacity=1,
        layer='below',
        line_width=0
    )

fig.show()


In [ ]:
# Visualizacion factor de potencia

import plotly.graph_objects as go
import pandas as pd

df_all['fp'] = df_all['p_activa_total'] / np.sqrt(
    df_all['p_activa_total']**2 + df_all['q_reactiva_total']**2    
)

# Horarios
fecha = df_all['temporal_placa'].dt.normalize().min()

horarios = {
    'TURNO_MAÑANA': (fecha + pd.Timedelta(hours=5),
                     fecha + pd.Timedelta(hours=17)),
    'ALMUERZO':     (fecha + pd.Timedelta(hours=12),
                     fecha + pd.Timedelta(hours=12, minutes=30)),
    'TURNO_TARDE':  (fecha + pd.Timedelta(hours=17),
                     fecha + pd.Timedelta(hours=22)),
    'CENA':         (fecha + pd.Timedelta(hours=22),
                     fecha + pd.Timedelta(hours=22, minutes=30)),
    'FUERA_TURNO':  (fecha + pd.Timedelta(hours=1),
                     fecha + pd.Timedelta(hours=5))
}

colores_turnos = {
    'TURNO_MAÑANA': 'rgba(52, 101, 164, 0.25)',
    'TURNO_TARDE':  'rgba(46, 139, 87, 0.25)',
    'ALMUERZO':     'rgba(218, 165, 32, 0.30)',
    'CENA':         'rgba(178, 34, 34, 0.30)',
    'FUERA_TURNO':  'rgba(105, 105, 105, 0.20)'
}

maquinas = list(df_all['maquina'].unique())

# Figura y trazas
fig = go.Figure()

labels = []  # para el dropdown

for maq in maquinas:
    g = df_all[df_all['maquina'] == maq]
    labels.append(f'{maq} – {fase_label}')

    fig.add_trace(
           go.Scatter(
            x=g['temporal_placa'],
            y=g['fp'],
            mode='lines',
            name=f'{maq}',
            visible=False
        )
    )

# Estado inicial
fig.data[0].visible = True

# Dropdown único
buttons = []

for i, label in enumerate(labels):
    visible = [False] * len(fig.data)
    visible[i] = True

    buttons.append(
        dict(
            label=label,
            method='update',
            args=[
                {'visible': visible},
                {'title.text': f'Corriente vs tiempo – {label}'}
            ]
        )
    )

# Layout
fig.update_layout(
    title=dict(
        text=f'Factor de Potencia vs tiempo – {labels[0]}',
        x=0.5,
        xanchor='center',
        font=dict(size=18)
    ),
    yaxis_title='Corriente [A]',
    showlegend=False,
    margin=dict(r=260),
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=1.02,
            y=1.00,
            xanchor='left',
            yanchor='top'
        )
    ]
)

# Bandas horarias
for nombre, (inicio, fin) in horarios.items():
    fig.add_vrect(
        x0=inicio,
        x1=fin,
        fillcolor=colores_turnos[nombre],
        opacity=1,
        layer='below',
        line_width=0
    )

fig.show()


In [12]:
# Visualizacion fase mas cargada + franjas de trabajo
import plotly.graph_objects as go
import pandas as pd

# Determinar fase más cargada por MEDIA (por máquina)
medias_por_fase = (
    df_all
    .groupby('maquina')
    .agg(
        I_R_media=('corriente_r', 'mean'),
        I_S_media=('corriente_s', 'mean'),
        I_T_media=('corriente_t', 'mean')
    )
)

medias_por_fase['fase_mas_cargada'] = medias_por_fase[
    ['I_R_media', 'I_S_media', 'I_T_media']
].idxmax(axis=1)

map_fase_col = {
    'I_R_media': 'corriente_r',
    'I_S_media': 'corriente_s',
    'I_T_media': 'corriente_t'
}

map_fase_label = {
    'I_R_media': 'Fase R',
    'I_S_media': 'Fase S',
    'I_T_media': 'Fase T'
}

# 2. Figuras (una por máquina)
figs = {}
maquinas = medias_por_fase.index.tolist()

for maq in maquinas:

    fase_media = medias_por_fase.loc[maq, 'fase_mas_cargada']
    fase_col = map_fase_col[fase_media]
    fase_label = map_fase_label[fase_media]

    g_elec = df_all[df_all['maquina'] == maq]
    g_ciclos = df_ciclos[df_ciclos['maquina'] == maq]

    fig = go.Figure()

    # Señal eléctrica: fase más cargada
    fig.add_trace(
        go.Scatter(
            x=g_elec['temporal_placa'],
            y=g_elec[fase_col],
            mode='lines',
            line=dict(color='#1f77b4', width=2),
            name=f'{maq} – {fase_label}',
            hovertemplate=(
                'Tiempo: %{x}<br>'
                'Corriente RMS: %{y:.1f} A<br>'
                f'{fase_label}<extra></extra>'
            )
        )
    )

    # Bandas de ciclos + líneas inicio/fin
    for _, row in g_ciclos.iterrows():

        # Banda del ciclo
        fig.add_vrect(
            x0=row['fecha_inicio'],
            x1=row['fecha_fin'],
            fillcolor='rgba(0,0,0,0.06)',
            opacity=1,
            layer='above',
            line_width=0
        )

        # Línea inicio
        fig.add_vline(
            x=row['fecha_inicio'],
            line_color='rgba(0,0,0,0.25)',
            line_dash='dot',
            line_width=1
        )

        # Línea fin
        fig.add_vline(
            x=row['fecha_fin'],
            line_color='rgba(0,0,0,0.25)',
            line_dash='dot',
            line_width=1
        )

    # Layout
    fig.update_layout(
        title=dict(
            text=f'{maq} – {fase_label} (fase más cargada por media)',
            x=0.5,
            xanchor='center',
            font=dict(size=18)
        ),
        xaxis_title='Tiempo',
        yaxis_title='Corriente RMS [A]',
        showlegend=False
    )

    figs[maq] = fig

# Dropdown para seleccionar máquina
fig_master = go.Figure()

for i, maq in enumerate(maquinas):
    for tr in figs[maq].data:
        tr.visible = (i == 0)
        fig_master.add_trace(tr)

fig_master.layout.shapes = figs[maquinas[0]].layout.shapes

buttons = []

offset = 0
for i, maq in enumerate(maquinas):
    n_traces = len(figs[maq].data)

    visible = [False] * len(fig_master.data)
    for j in range(offset, offset + n_traces):
        visible[j] = True

    buttons.append(
        dict(
            label=maq,
            method='update',
            args=[
                {'visible': visible},
                {
                    'title.text': figs[maq].layout.title.text,
                    'shapes': figs[maq].layout.shapes
                }
            ]
        )
    )

    offset += n_traces

fig_master.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=1.02,
            y=1.0,
            xanchor='left',
            yanchor='top'
        )
    ],
    margin=dict(r=260)
)

fig_master.show()


In [22]:
# Funcion detectar_picos
def detectar_picos(signal, threshold):
    """
    signal: pd.Series
    threshold: float
    Devuelve índices de picos
    """
    return signal[
        (signal > threshold) &
        (signal.shift(1) < signal) &
        (signal.shift(-1) < signal)
    ].index


In [25]:
# Deteccion de picos de corriente (por máquina)
# usando la FASE MÁS CARGADA POR MEDIA
df_peaks_list = []

for maq in maquinas:

    # fase más cargada
    fase_media = medias_por_fase.loc[maq, 'fase_mas_cargada']
    fase_col = map_fase_col[fase_media]

    g = (
        df_all[df_all['maquina'] == maq]
        .sort_values('temporal_placa')
        .copy()
    )

    señal = g[fase_col]

    media = señal.mean()
    umbral = 1.1 * media   # +20% sobre la media

    idx_picos = detectar_picos(señal, umbral)

    df_peaks = g.loc[idx_picos, ['temporal_placa']].copy()
    df_peaks['maquina'] = maq
    df_peaks['fase'] = map_fase_label[fase_media]

    df_peaks_list.append(df_peaks)

df_peaks_all = pd.concat(df_peaks_list, ignore_index=True)

df_peaks_all['temporal_placa'] = (
    pd.to_datetime(df_peaks_all['temporal_placa'], errors='coerce')
    .dt.tz_localize(None)
)

df_ciclos['picos_detectados'] = 0

for idx, ciclo in df_ciclos.iterrows():

    maq = ciclo['maquina']
    t_ini = ciclo['fecha_inicio']
    t_fin = ciclo['fecha_fin']

    picos = df_peaks_all[
        (df_peaks_all['maquina'] == maq) &
        (df_peaks_all['temporal_placa'] >= t_ini) &
        (df_peaks_all['temporal_placa'] <= t_fin)
    ]

    df_ciclos.at[idx, 'picos_detectados'] = len(picos)

#df_ciclos


In [26]:
# Visualizacion de picos detectados y su marca temporal

df_peaks_all = df_peaks_all.sort_values(
    by=['maquina', 'temporal_placa']
).reset_index(drop=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df_peaks_all

,temporal_placa,maquina,fase
0,2026-01-09 16:57:16,Plegadora 1,Fase S
1,2026-01-09 16:57:39,Plegadora 1,Fase S
2,2026-01-09 16:57:49,Plegadora 1,Fase S
3,2026-01-09 16:58:04,Plegadora 1,Fase S
4,2026-01-09 17:04:56,Plegadora 1,Fase S
5,2026-01-09 17:05:06,Plegadora 1,Fase S
6,2026-01-09 17:05:41,Plegadora 1,Fase S
7,2026-01-09 17:11:22,Plegadora 1,Fase S
8,2026-01-09 17:11:30,Plegadora 1,Fase S
9,2026-01-09 17:11:45,Plegadora 1,Fase S


In [39]:
# Visualizacion Tensiones
import plotly.graph_objects as go
import pandas as pd
fecha = df_all['temporal_placa'].dt.date.min()

horarios = {
    'TURNO_MAÑANA': (f'{fecha} 05:00', f'{fecha} 17:00'),
    'ALMUERZO':     (f'{fecha} 12:00', f'{fecha} 12:30'),
    'TURNO_TARDE':  (f'{fecha} 17:00', f'{fecha} 22:00'),
    'CENA':         (f'{fecha} 22:00', f'{fecha} 22:30'),
    'FUERA_TURNO':  (f'{fecha} 01:00', f'{fecha} 05:00')
}

colores_turnos = {
    'TURNO_MAÑANA': 'rgba(52, 101, 164, 0.25)',   # azul acero
    'TURNO_TARDE':  'rgba(46, 139, 87, 0.25)',    # verde bosque
    'ALMUERZO':     'rgba(218, 165, 32, 0.30)',   # dorado apagado
    'CENA':         'rgba(178, 34, 34, 0.30)',    # rojo ladrillo
    'FUERA_TURNO':  'rgba(105, 105, 105, 0.20)'   # gris antracita
}

horarios = {
    k: (pd.to_datetime(v[0]), pd.to_datetime(v[1]))
    for k, v in horarios.items()
}

# Definición de fases
fases_tension = {
    'Fase R': 'tension_r',
    'Fase S': 'tension_s',
    'Fase T': 'tension_t'
}


maquinas = df_all['maquina'].unique()

fig = go.Figure()

# Colores por máquina (los mismos que corrientes)
colores_maquina = {
    'Plegadora 1': '#1f77b4',  # azul
    'Plegadora 7': '#d62728',  # rojo
}

maquinas = df_all['maquina'].unique()


# Trazas: fase × máquina (TENSION)
for fase_label, fase_col in fases_tension.items():
    for maq in maquinas:
        g = df_all[df_all['maquina'] == maq]

        fig.add_trace(
            go.Scatter(
                x=g['temporal_placa'],
                y=g[fase_col],
                mode='lines',
                name=f'{maq}',
                line=dict(
                    color=colores_maquina.get(maq, '#7f7f7f'),
                    width=2
                ),
                visible=(fase_label == 'Fase R')  # arranca en Fase R
            )
        )

# Dropdown de fases
buttons = []
n_maquinas = len(maquinas)

for i, fase_label in enumerate(fases_tension.keys()):
    visible = [False] * len(fig.data)

    for j in range(n_maquinas):
        visible[i * n_maquinas + j] = True

    buttons.append(
        dict(
            label=fase_label,
            method='update',
            args=[
                {'visible': visible},
                {
                    'title': {
                        'text': f'Tensión vs tiempo – {fase_label}',
                        'x': 0.5,
                        'xanchor': 'center',
                        'font': {'size': 18}
                    }
                }
            ]
        )
    )

# Layout
fig.update_layout(
    title=dict(
        text='Tensión vs tiempo – Fase R',
        x=0.5,
        xanchor='center',
        font=dict(size=18)
    ),
    yaxis_title='Tensión [V]',
    xaxis_title='Tiempo',
    legend_title='Máquina',
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=0.01,
            y=1.15
        )
    ]
)

# Bandas de turnos y pausas
for nombre, (inicio, fin) in horarios.items():
    fig.add_vrect(
        x0=inicio,
        x1=fin,
        fillcolor=colores_turnos.get(nombre, 'rgba(150,150,150,0.05)'),
        opacity=1,
        layer='below',
        line_width=0,
        annotation_text=nombre.replace('_', ' '),
        annotation_position='bottom left'
    )

fig.show()


In [ ]:
# Calculo y grafica de la potencia aparente total
# Stotal = Ir.Vr + Is.Vs + It.Vt
import plotly.graph_objects as go
import pandas as pd

df_all['S_total_kVA'] = (
    df_all['tension_r'] * df_all['corriente_r'] +
    df_all['tension_s'] * df_all['corriente_s'] +
    df_all['tension_t'] * df_all['corriente_t']
) / 1000

# Asegurar fechas sin timezone
df_all['temporal_placa'] = pd.to_datetime(df_all['temporal_placa'], errors='coerce').dt.tz_localize(None)
df_ciclos['fecha_inicio'] = pd.to_datetime(df_ciclos['fecha_inicio'], errors='coerce').dt.tz_localize(None)
df_ciclos['fecha_fin']    = pd.to_datetime(df_ciclos['fecha_fin'],    errors='coerce').dt.tz_localize(None)


# Construir una figura por máquina (con sus bandas)
figs = {}
maquinas = df_all['maquina'].unique().tolist()

for maq in maquinas:
    g_elec   = df_all[df_all['maquina'] == maq].sort_values('temporal_placa')
    g_ciclos = df_ciclos[df_ciclos['maquina'] == maq]

    fig = go.Figure()

    # Señal eléctrica: kVA
    fig.add_trace(
        go.Scatter(
            x=g_elec['temporal_placa'],
            y=g_elec['S_total_kVA'],
            mode='lines',
            line=dict(width=2),
            name=f'{maq} – kVA',
            hovertemplate=(
                'Tiempo: %{x}<br>'
                'Potencia aparente: %{y:.1f} kVA<extra></extra>'
            )
        )
    )

    # Bandas de ciclos + líneas inicio/fin
    for _, row in g_ciclos.iterrows():
        # Banda
        fig.add_vrect(
            x0=row['fecha_inicio'],
            x1=row['fecha_fin'],
            fillcolor='rgba(0,0,0,0.06)',
            opacity=1,
            layer='above',
            line_width=0
        )
        # Línea inicio
        fig.add_vline(
            x=row['fecha_inicio'],
            line_color='rgba(0,0,0,0.25)',
            line_dash='dot',
            line_width=1
        )
        # Línea fin
        fig.add_vline(
            x=row['fecha_fin'],
            line_color='rgba(0,0,0,0.25)',
            line_dash='dot',
            line_width=1
        )

    fig.update_layout(
        title=dict(
            text=f'{maq} – Potencia aparente total (kVA) + ciclos',
            x=0.5,
            xanchor='center',
            font=dict(size=18)
        ),
        xaxis_title='Tiempo',
        yaxis_title='Potencia aparente [kVA]',
        showlegend=False
    )

    figs[maq] = fig

# Figura maestra con dropdown (elige máquina)
fig_master = go.Figure()

# Agregar todas las trazas (una máquina visible al inicio)
for i, maq in enumerate(maquinas):
    for tr in figs[maq].data:
        tr.visible = (i == 0)
        fig_master.add_trace(tr)

# Shapes iniciales (de la primera máquina)
fig_master.layout.shapes = figs[maquinas[0]].layout.shapes

# Dropdown
buttons = []
offset = 0
for i, maq in enumerate(maquinas):
    n_traces = len(figs[maq].data)
    visible = [False] * len(fig_master.data)
    for j in range(offset, offset + n_traces):
        visible[j] = True

    buttons.append(
        dict(
            label=maq,
            method='update',
            args=[
                {'visible': visible},
                {
                    'title.text': figs[maq].layout.title.text,
                    'shapes': figs[maq].layout.shapes
                }
            ]
        )
    )
    offset += n_traces

fig_master.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=1.02,
            y=1.0,
            xanchor='left',
            yanchor='top'
        )
    ],
    margin=dict(r=260)
)

fig_master.show()

